In [2]:
import numpy as np
import pandas as pd
import altair as alt
from vega_datasets import data as vega_data

# Top 10 Tornados
Allows user to explore their own interests in tornado severity among the four categories present in the data
* Magnitude
* Fatality
* Injury
* Property Loss
* Crop Loss


In [3]:
# Prepare data
data = pd.read_csv("Tornado_Tracks.csv")
data_clean = data[["OBJECTID", "date", "st", "mag", "inj", "fat", "loss", "closs"]]
data_clean = data_clean.rename(columns={"OBJECTID": "id", "date": "Date", "st": "State", "mag": "Magnitude", "inj": "Injuries", "fat": "Fatalities", "loss": "Property_Loss", "closs": "Crop_Loss"})
data_clean.head(10)

# Filter data for the different top 10s -- bypass the too large data
# magnitude = data_clean.nlargest(10, "Magnitude")
injuries = data_clean.nlargest(10, "Injuries")
fatalities = data_clean.nlargest(10, "Fatalities")
losses = data_clean.nlargest(10, "Property_Loss")
crop_losses = data_clean.nlargest(10, "Crop_Loss")

# Adding a metric column for each data set
# magnitude["Metric"] = "Magnitude"
injuries["Metric"] = "Injuries"
fatalities["Metric"] = "Fatalities"
losses["Metric"] = "Property Loss ($M)"
crop_losses["Metric"] = "Crop Loss ($M)"

# Flatten the data
# magnitude["Value"] = magnitude["Magnitude"]
injuries["Value"] = injuries["Injuries"]
fatalities["Value"] = fatalities["Fatalities"]
losses["Value"] = losses["Property_Loss"] / 1000000  # Convert to millions
crop_losses["Value"] = crop_losses["Crop_Loss"] / 1000000  # Convert to millions

# Join Charts
top10_combined = pd.concat([
#     magnitude[["id", "Date", "State", "Metric", "Value"]],
    injuries[["id", "Date", "State", "Metric", "Value"]],
    fatalities[["id", "Date", "State", "Metric", "Value"]],
    losses[["id", "Date", "State", "Metric", "Value"]],
    crop_losses[["id", "Date", "State", "Metric", "Value"]]
], ignore_index=True)

top10_combined

,id,Date,State,Metric,Value
0,19483,1979/04/10,TX,Injuries,1740.000
1,55297,2011/04/27,AL,Injuries,1500.000
2,1058,1953/06/09,MA,Injuries,1228.000
3,15260,1974/04/03,OH,Injuries,1150.000
4,55601,2011/05/22,MO,Injuries,1150.000
5,1055,1953/06/08,MI,Injuries,844.000
6,8104,1965/04/11,IN,Injuries,835.000
7,12412,1971/02/21,MS,Injuries,795.000
8,882,1953/05/11,TX,Injuries,597.000
9,39850,1999/05/03,OK,Injuries,583.000


In [16]:
# Have interaction with the legend
metrics = top10_combined['Metric'].unique()

# Create drop down for the metric
# Create a selection tied to the dropdown menu
metric_param=alt.selection_point(
    fields=['Metric'],
    value=metrics[0],
    bind=alt.binding_select(options=metrics, name="Select Metric"),
)

top_10 = alt.Chart(top10_combined).transform_filter(
    metric_param
).mark_circle(stroke='black', strokeWidth=0.5, size=40).encode(
    x = alt.X('yearmonth(Date):T', title = "Date", scale=alt.Scale(range=[1953, 2025])).axis(domain=False, format='%Y ', tickSize=0),
    y = alt.Y('Value:Q'),
    size = alt.Size('Value:Q', scale=alt.Scale(range=[0, 500])),
    color = alt.Color('Metric:N', scale=alt.Scale(domain = metrics, range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])),
    tooltip=[
        alt.Tooltip('Date:T'),
        alt.Tooltip('State:N'),
        alt.Tooltip('Metric:N'),
        alt.Tooltip('Value:Q'),
        alt.Tooltip('id:O')
    ]
).add_params(
    metric_param
).configure_title(anchor='middle',
                                fontSize=30,
                                font='Helvetica',
                                color='navy'
                            ).configure_view(
                                fill='white'
                            ).properties(
                                background='#e6f7ff',# light pastel blue
                                title = "Top 10 Tornadoes in the US by Metric",
                                width=400,
                                height = 300
                            )

# Save the chart
top_10.save('top_10_tornadoes.html')

# Mobile Homes 

In [5]:
# Load data
mobile_home_data = pd.read_csv("Mobile_Home_Parks.csv")

# filter for homes that are active
mobile_home_data = mobile_home_data[mobile_home_data.STATUS == "Open"]

# Cols to keep: X, Y, STATE, TYPE, LATITUDE, LONGITUDE, NAICS_DESC, UNITS, SIZE
mh_clean = mobile_home_data[["STATE", "COUNTYFIPS","TYPE", "LATITUDE", "LONGITUDE", "NAICS_DESC", "UNITS"]]
mh_clean

# State FIPS
# mh_clean["FIPS"] = mh_clean["COUNTYFIPS"].astype(str).str.slice(0, 2)
mh_clean["FIPS"] = mh_clean["COUNTYFIPS"].astype(str).str.zfill(5).str.slice(0, 2)

mh_clean.head()


/var/folders/w7/0v1gkzq5479720zd0j6c4p2m0000gn/T/ipykernel_35424/4159283866.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mh_clean["FIPS"] = mh_clean["COUNTYFIPS"].astype(str).str.zfill(5).str.slice(0, 2)


,STATE,COUNTYFIPS,TYPE,LATITUDE,LONGITUDE,NAICS_DESC,UNITS,FIPS
0,FL,12069,Mobile Home Park,28.852114,-81.687714,Residential trailer parks,0,12
1,FL,12063,Mobile Home Park,30.771086,-85.232052,Residential trailer parks,0,12
2,FL,12063,Mobile Home Park,30.789739,-85.163627,Residential trailer parks,0,12
3,FL,12069,Mobile Home Park,28.815828,-81.687733,Mobile (manufactured) home parks,0,12
4,FL,12069,Mobile Home Park,28.529614,-81.784127,Mobile (manufactured) home parks,0,12


## Choropleth Map

In [6]:
# Gather the data by state
mh_subset = mh_clean[["STATE", "FIPS"]]

mh_subset_aggregate = mh_subset.groupby(["STATE", "FIPS"]).size().reset_index(name='COUNT')
mh_filter = mh_subset_aggregate[mh_subset_aggregate["COUNT"] > 1]

# fips code for california 
mh_filter['FIPS'] = mh_filter['FIPS'].astype(int)

/var/folders/w7/0v1gkzq5479720zd0j6c4p2m0000gn/T/ipykernel_35424/3453362447.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mh_filter['FIPS'] = mh_filter['FIPS'].astype(int)


In [11]:
# Geographic choropleth map
# Number of mobile homes located in each state
selection = alt.selection_point(fields=['STATE'], name="Select State")
states = alt.topo_feature(vega_data.us_10m.url, 'states')

map_chart = alt.Chart(states).mark_geoshape().encode(
    color=alt.Color('COUNT:Q', scale=alt.Scale(scheme='blues')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=["COUNT:Q", "STATE:N"]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(mh_filter, 'FIPS', ['STATE', 'COUNT'])
).add_params(
    selection
).project(
    'albersUsa'
).properties(
    title='Mobile Home Parks Distribution in the US',
    width=400
)

map_chart

bar_chart = alt.Chart(mh_filter).mark_bar().encode(
    x=alt.X('COUNT:Q', title='Mobile Home Count'),
    y=alt.Y('STATE:N', sort='-x', title='State'),
    color=alt.Color('COUNT:Q', scale=alt.Scale(scheme='blues')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).transform_window(
    rank='rank(COUNT)',
    sort=[alt.SortField('COUNT', order='descending')]
).transform_filter(
    alt.datum.rank <= 15
).add_params(
    selection
).properties(
    title = 'Top 15 States by Mobile Home Parks',
    width=400)

combined_chart = alt.hconcat(map_chart, bar_chart
                             ).configure_title(anchor='middle',
                                fontSize=20,
                                font='Helvetica',
                                color='navy'
                            ).configure_view(
                                fill='white'
                            ).properties(
                                background='#e6f7ff'# light pastel blue
                            )
combined_chart.save("choropleth_Mobile_Homes.html")
combined_chart

alt.HConcatChart(...)